In [37]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
import torch
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertModel, BertTokenizer
from math import radians, cos, sin, asin, sqrt, atan2


In [2]:
df = pd.read_json('All_Category_embeddings.json')

In [3]:
df.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,category_embeddings
0,St Peter & Paul Church,"St Peter & Paul Church, 500 Main St, Karlsruhe...",0x52d94fbefa0e6353:0xf709e2d8674fe3a,None,48.093248,-100.618664,Catholic church,4.9,7,None,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x52d9384b75abac93:0x13526f8266cae6cf, 0x52d9...",https://www.google.com/maps/place//data=!4m2!3...,"[0.2593699098, 0.0440241471, -0.35110336540000..."
1,Northwest Martial Arts Academy,"Northwest Martial Arts Academy, 1430 Main Ave,...",0x52c8cbe775edec7d:0xb46e15ed33643070,None,46.875093,-96.802717,Martial arts school,5.0,8,None,"[[Thursday, 7:30AM–8PM], [Friday, 7:30AM–8PM],...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 7:30AM,"[0x52c8ccbcb1785327:0x2d50311eabd7afc, 0x52cf3...",https://www.google.com/maps/place//data=!4m2!3...,"[0.0723896474, -0.1581346691, -0.2116216719, -..."
2,Thad's Amazing Magic - Fargo Birthday Party Magic,Thad's Amazing Magic - Fargo Birthday Party Ma...,0x52c8cd270f50bbbb:0x4ee4629598a8090e,None,46.812415,-96.856729,Magician,5.0,58,None,"[[Thursday, Open 24 hours], [Friday, Open 24 h...",{'Amenities': ['Good for kids']},Open 24 hours,"[0x52c8c9613725e9ef:0xc628b86d8593e7e6, 0x52c8...",https://www.google.com/maps/place//data=!4m2!3...,"[0.0990275964, 0.12004911900000001, -0.3748326..."
3,Thad's Amazing Magic - Fargo Birthday Party Magic,Thad's Amazing Magic - Fargo Birthday Party Ma...,0x52c8cd270f50bbbb:0x4ee4629598a8090e,None,46.812415,-96.856729,Children's party service,5.0,58,None,"[[Thursday, Open 24 hours], [Friday, Open 24 h...",{'Amenities': ['Good for kids']},Open 24 hours,"[0x52c8c9613725e9ef:0xc628b86d8593e7e6, 0x52c8...",https://www.google.com/maps/place//data=!4m2!3...,"[-0.1832211912, -0.401704371, -0.1263380498, -..."
4,Threefold,"Threefold, 212 W Main Ave, Bismarck, ND 58501",0x52d7836b7314da5d:0xc3cc63667b8c13a0,None,46.805707,-100.792990,Film production company,5.0,5,None,"[[Wednesday, 9AM–6PM], [Thursday, 9AM–6PM], [F...",None,Closed ⋅ Opens 9AM Thu,"[0x52d7836c2b519b77:0x74c84187e38f42b, 0x52d78...",https://www.google.com/maps/place//data=!4m2!3...,"[0.3351577818, 0.23361778260000002, -0.4228747..."


In [4]:
df.shape

(27057, 16)

In [34]:
user_long = -96.611273
user_lat = 46.268488

In [5]:
# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
model = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)


In [16]:
def convert_user_input(user_input):
    tokens = tokenizer(user_input, truncation=True, padding=True, max_length=128, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**tokens)

    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

In [32]:
# Caculating distance of business from the user
def check_business_distance(long1, lat1, long2, lat2):
    long1 = radians(long1)
    lat1 = radians(lat1)
    long2 = radians(long2)
    lat2 = radians(lat2)
    
    dlon = long2 - long1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2

    c = 2 * asin(sqrt(a))

    r = 6371 # Radius of earth in kilometers

    return (c * r)

In [33]:
# Sorting similar businesses by their distance and ratings
def process_related_business(df):
    df['business_distance'] = [check_business_distance(user_long, user_lat, long, lat) for long, lat in zip(df['longitude'], df['latitude'])]
    df = df.sort_values(by=['business_distance', 'avg_rating', 'num_of_reviews'], ascending=[True, False, False])
    return df

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

# Find most related business as per user search
def find_business(user_input):
    user_vector = convert_user_input(user_input)
    similar_business = []
    for categories in df['category_embeddings']:
        # Ensure both user_vector and categories are 2D arrays
        user_vector_array = np.array(user_vector).reshape(1, -1)
        categories_array = np.array(categories).reshape(1, -1)
        
        cosine_similarities = cosine_similarity(user_vector_array, categories_array)
        similar_business.append(cosine_similarities.max())

    similar_business = np.where(np.array(similar_business) >= 0.8)[0]
    similar_business_df = df.iloc[similar_business]
    similar_business_df = process_related_business(similar_business_df)
    return similar_business_df

In [28]:
df['category_embeddings'].isnull().sum()

0

In [27]:
df = df.dropna(subset=['category_embeddings'])

In [38]:
df_ans = find_business('Restaurant')
df_ans.head(5)

C:\Users\nishi\AppData\Local\Temp\ipykernel_16300\1146921267.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['business_distance'] = [check_business_distance(user_long, user_lat, long, lat) for long, lat in zip(df['longitude'], df['latitude'])]


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,category_embeddings,business_distance
21806,"Econo Wine & Spirits,",None,0x52c9493b483b1a8b:0x347b78195c676759,None,46.265969,-96.612918,Wine store,4.2,18,None,"[[Friday, 8AM–12AM], [Saturday, 8AM–12AM], [Su...",{'Payments': ['NFC mobile payments']},Closed ⋅ Opens 8AM,"[0x52c948c82ae3dfcf:0x271bc92e1ade5076, 0x52c9...",https://www.google.com/maps/place//data=!4m2!3...,"[0.3938874006, -0.07238018510000001, -0.244404...",0.307332
4477,Johnson Law Office,"Johnson Law Office, 205 7th St N, Wahpeton, ND...",0x52c9492179915ced:0xd3fb53f31fa5eee7,None,46.265382,-96.610078,Attorney,4.5,8,None,None,"{'Health & safety': ['Appointment required'], ...",None,"[0x52c948dece2260db:0x3de5de8f1bef432c, 0x52c9...",https://www.google.com/maps/place//data=!4m2!3...,"[0.23098389800000002, 0.193268761, -0.19212853...",0.357346
4996,"Lies, Bullis & Hatting, PLLP","Lies, Bullis & Hatting, PLLP, 610 2nd Ave N, W...",0x52c949210edcc25f:0x8db03e222cd5436d,None,46.265260,-96.608903,Attorney,3.7,8,None,"[[Wednesday, 8AM–5PM], [Thursday, 8AM–5PM], [F...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 8AM,"[0x52c949215effcd47:0x1127c0e48eae216, 0x52c94...",https://www.google.com/maps/place//data=!4m2!3...,"[0.23098389800000002, 0.193268761, -0.19212853...",0.402514
10283,Family Dollar,"Family Dollar, 824 Dakota Ave, Wahpeton, ND 58075",0x52c949239542b547:0x9196db4fe922d519,"Discount chain carrying a variety of goods, in...",46.264725,-96.612406,Dollar store,3.6,65,$,"[[Sunday, 9AM–9PM], [Monday, 9AM–9PM], [Tuesda...","{'Service options': ['In-store shopping', 'Del...",Closed ⋅ Opens 9AM,"[0x52c9495776f30e7b:0x75494f25a706f6f, 0x52c94...",https://www.google.com/maps/place//data=!4m2!3...,"[0.47631388900000005, -0.1689469963, -0.226667...",0.427334
10284,Family Dollar,"Family Dollar, 824 Dakota Ave, Wahpeton, ND 58075",0x52c949239542b547:0x9196db4fe922d519,"Discount chain carrying a variety of goods, in...",46.264725,-96.612406,Clothing store,3.6,65,$,"[[Sunday, 9AM–9PM], [Monday, 9AM–9PM], [Tuesda...","{'Service options': ['In-store shopping', 'Del...",Closed ⋅ Opens 9AM,"[0x52c9495776f30e7b:0x75494f25a706f6f, 0x52c94...",https://www.google.com/maps/place//data=!4m2!3...,"[0.49196720120000004, -0.1564666629, -0.259400...",0.427334
